In [2]:
import os
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
STUDY_PATH=os.path.join("/NAS","dumbo","protocoles","meningiome_AZ","test_perf","DICOM")